<a href="https://colab.research.google.com/github/Emmanuel2000x/TelecomX/blob/main/TelecomX_LATAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📌 Extracción

In [406]:
#Se importan las librerias necesarias para el proyecto
import pandas as pd
import numpy as np

In [407]:
#Se le la URL del .json con la informacion que otorga la API
url = "https://raw.githubusercontent.com/ingridcristh/challenge2-data-science-LATAM/refs/heads/main/TelecomX_Data.json"
datos_json = pd.read_json(url)

In [408]:
#El DF original cuenta 7267 registros con 6 columnas |customerID|Churn|customer|phone|internet |account|
print(datos_json.shape)
print(datos_json.head(5))

(7267, 6)
   customerID Churn                                           customer  \
0  0002-ORFBO    No  {'gender': 'Female', 'SeniorCitizen': 0, 'Part...   
1  0003-MKNFE    No  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
2  0004-TLHLJ   Yes  {'gender': 'Male', 'SeniorCitizen': 0, 'Partne...   
3  0011-IGKFF   Yes  {'gender': 'Male', 'SeniorCitizen': 1, 'Partne...   
4  0013-EXCHZ   Yes  {'gender': 'Female', 'SeniorCitizen': 1, 'Part...   

                                             phone  \
0   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
1  {'PhoneService': 'Yes', 'MultipleLines': 'Yes'}   
2   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
3   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   
4   {'PhoneService': 'Yes', 'MultipleLines': 'No'}   

                                            internet  \
0  {'InternetService': 'DSL', 'OnlineSecurity': '...   
1  {'InternetService': 'DSL', 'OnlineSecurity': '...   
2  {'InternetService': 'Fiber optic', 'OnlineSecu...

In [409]:
#Creamos una funcion encargada de normalizar todas las columnas anidadas
def normalize_nested_df(df: pd.DataFrame) -> pd.DataFrame:
  """
  Normaliza de forma recursiva las columnas anidadas de un DF
  Args:
    df (pd.DataFrame): Input DataFrame with potential nested structures.
  Returns:
    pd.DataFrame: Fully normalized (flattened) DataFrame.
  """
  df = df.copy()
  new_columns = []
  for col in df.columns:

    if df[col].apply(lambda x: isinstance(x, dict)).any():
      # Expand dictionaries into separate columns
      expanded = pd.json_normalize(df[col]).add_prefix(f'{col}_')
      df = df.drop(columns=[col])
      df = pd.concat([df, expanded], axis=1)
      return normalize_nested_df(df)  # Recursively normalize
    else:
      new_columns.append(col)

  return df


In [410]:
#Normalizacion del DF utilizando la funcion creada
flatt_DF = normalize_nested_df(datos_json)
#Ahora tenemos 21 columnas y ninguna con info anidada
print(flatt_DF.shape)
print(flatt_DF.head(3))

(7267, 21)
   customerID Churn customer_gender  customer_SeniorCitizen customer_Partner  \
0  0002-ORFBO    No          Female                       0              Yes   
1  0003-MKNFE    No            Male                       0               No   
2  0004-TLHLJ   Yes            Male                       0               No   

  customer_Dependents  customer_tenure phone_PhoneService phone_MultipleLines  \
0                 Yes                9                Yes                  No   
1                  No                9                Yes                 Yes   
2                  No                4                Yes                  No   

  internet_InternetService  ... internet_OnlineBackup  \
0                      DSL  ...                   Yes   
1                      DSL  ...                    No   
2              Fiber optic  ...                    No   

  internet_DeviceProtection internet_TechSupport internet_StreamingTV  \
0                        No              

##Lista de columnas y su clasificacion

In [411]:
#Lista que almacena las Variables/Columnas que contiene el DF
Datos_dic = list(flatt_DF.columns)
print(Datos_dic)

['customerID', 'Churn', 'customer_gender', 'customer_SeniorCitizen', 'customer_Partner', 'customer_Dependents', 'customer_tenure', 'phone_PhoneService', 'phone_MultipleLines', 'internet_InternetService', 'internet_OnlineSecurity', 'internet_OnlineBackup', 'internet_DeviceProtection', 'internet_TechSupport', 'internet_StreamingTV', 'internet_StreamingMovies', 'account_Contract', 'account_PaperlessBilling', 'account_PaymentMethod', 'account_Charges.Monthly', 'account_Charges.Total']


In [412]:
#Clasificamos el tipo de dato de cada columna
Col_Bol = ['Churn', 'customer_SeniorCitizen','customer_Partner',
        'customer_Dependents','phone_PhoneService', 'phone_MultipleLines',
        'internet_OnlineSecurity', 'internet_OnlineBackup','internet_DeviceProtection', 'internet_TechSupport',
        'internet_StreamingTV', 'internet_StreamingMovies','account_PaperlessBilling']
Col_Str = ['customerID','customer_gender', 'internet_InternetService',
           'account_Contract','account_PaymentMethod']
Col_Flo = ['account_Charges.Monthly', 'account_Charges.Total']
Col_Int = ['customer_tenure']

#🔧 Transformación

## Conocemos el tipo de dato con el que viene cada columna


In [413]:
#Exploramos las columnas del data set y verificamos los tipos de datos
flatt_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7267 entries, 0 to 7266
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7267 non-null   object 
 1   Churn                      7267 non-null   object 
 2   customer_gender            7267 non-null   object 
 3   customer_SeniorCitizen     7267 non-null   int64  
 4   customer_Partner           7267 non-null   object 
 5   customer_Dependents        7267 non-null   object 
 6   customer_tenure            7267 non-null   int64  
 7   phone_PhoneService         7267 non-null   object 
 8   phone_MultipleLines        7267 non-null   object 
 9   internet_InternetService   7267 non-null   object 
 10  internet_OnlineSecurity    7267 non-null   object 
 11  internet_OnlineBackup      7267 non-null   object 
 12  internet_DeviceProtection  7267 non-null   object 
 13  internet_TechSupport       7267 non-null   objec

In [414]:
flatt_DF.dtypes

,0
customerID,object
Churn,object
customer_gender,object
customer_SeniorCitizen,int64
customer_Partner,object
customer_Dependents,object
customer_tenure,int64
phone_PhoneService,object
phone_MultipleLines,object
internet_InternetService,object


In [415]:
#De origen el data set parece no contener errores nulos
print(flatt_DF.isna().sum())

customerID                   0
Churn                        0
customer_gender              0
customer_SeniorCitizen       0
customer_Partner             0
customer_Dependents          0
customer_tenure              0
phone_PhoneService           0
phone_MultipleLines          0
internet_InternetService     0
internet_OnlineSecurity      0
internet_OnlineBackup        0
internet_DeviceProtection    0
internet_TechSupport         0
internet_StreamingTV         0
internet_StreamingMovies     0
account_Contract             0
account_PaperlessBilling     0
account_PaymentMethod        0
account_Charges.Monthly      0
account_Charges.Total        0
dtype: int64


In [416]:
#Replace de especios en blanco (e.g., '', ' ', '  ') with NaN
mod_DF = flatt_DF.replace(r'^\s*$', np.nan, regex=True)
#Lista de los indices donde hay valores NaN
error_indices = mod_DF[mod_DF.isna().any(axis=1)].index.tolist()

In [417]:
#Una vez transformados los datos detectamos lo siguiente
print(mod_DF.isna().sum())
print(f'Se encontraron {len(error_indices)} registros no validos,\nRepresentan el {100*len(error_indices)/len(flatt_DF.index):.2f}% del total')

customerID                     0
Churn                        224
customer_gender                0
customer_SeniorCitizen         0
customer_Partner               0
customer_Dependents            0
customer_tenure                0
phone_PhoneService             0
phone_MultipleLines            0
internet_InternetService       0
internet_OnlineSecurity        0
internet_OnlineBackup          0
internet_DeviceProtection      0
internet_TechSupport           0
internet_StreamingTV           0
internet_StreamingMovies       0
account_Contract               0
account_PaperlessBilling       0
account_PaymentMethod          0
account_Charges.Monthly        0
account_Charges.Total         11
dtype: int64
Se encontraron 235 registros no validos,
Representan el 3.23% del total


##Eliminado de registros sin info

In [418]:
"""
Dado que representan poca cantidad seran eliminados los registros
creando un nuevo data frame el cual contendra la transformacion
de la informacion
"""
mod_DF.dropna(inplace=True)
mod_DF.reset_index(drop=True,inplace=True)
print(mod_DF.shape)

(7032, 21)


In [419]:
#Comprobacion de la columna customerID
#Aseguramos el formato de los ID
mod_DF['customerID'] = mod_DF['customerID'].astype(str).str.upper()
#Buscamos registros duplicados
duplicados = mod_DF['customerID'].duplicated().any()
if not duplicados:
    print("No hay duplicados.")
else:
    print("Hay duplicados.")

No hay duplicados.


In [420]:
#Obvservamos los valores que contienen las otras columnas que no son numeros
for col in Datos_dic[1:-2]:
    print(f"Valores en {col}:")
    print(mod_DF[col].astype(str).unique())

Valores en Churn:
['No' 'Yes']
Valores en customer_gender:
['Female' 'Male']
Valores en customer_SeniorCitizen:
['0' '1']
Valores en customer_Partner:
['Yes' 'No']
Valores en customer_Dependents:
['Yes' 'No']
Valores en customer_tenure:
['9' '4' '13' '3' '71' '63' '7' '65' '54' '72' '5' '56' '34' '1' '45' '50'
 '23' '55' '26' '69' '37' '49' '66' '67' '20' '43' '59' '12' '27' '2' '25'
 '29' '14' '35' '64' '39' '40' '11' '6' '30' '70' '57' '58' '16' '32' '33'
 '10' '21' '61' '15' '44' '22' '24' '19' '47' '62' '46' '52' '8' '60' '48'
 '28' '41' '53' '68' '31' '36' '17' '18' '51' '38' '42']
Valores en phone_PhoneService:
['Yes' 'No']
Valores en phone_MultipleLines:
['No' 'Yes' 'No phone service']
Valores en internet_InternetService:
['DSL' 'Fiber optic' 'No']
Valores en internet_OnlineSecurity:
['No' 'Yes' 'No internet service']
Valores en internet_OnlineBackup:
['Yes' 'No' 'No internet service']
Valores en internet_DeviceProtection:
['No' 'Yes' 'No internet service']
Valores en internet_T

Con el analisis anterior encontramos que en la columna Churn hay registros en blanco, ademas en las columnas de servicios se presenta el "No", "No phone service" y "No internet service"

In [421]:
#convirtiendo el tipo de dato a float
mod_DF[Col_Flo] = mod_DF[Col_Flo].apply(pd.to_numeric, errors='coerce')

In [422]:
#Reemplazamos "Yes" y "No" por 1 y 0
mod_DF[Col_Bol] = mod_DF[Col_Bol].apply(lambda x: x.replace({
    'No': '0', 'No internet service': '0', 'No phone service': '0'})
    .replace('Yes','1'))
#Cambiamos el tipo de dato a booleano
mod_DF[Col_Bol] = mod_DF[Col_Bol].astype(int).astype(bool)

In [423]:
#Cambiamos el tipo de dato de las columnas de texto
mod_DF[Col_Str] = mod_DF[Col_Str].astype('string')

In [424]:
mod_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7032 entries, 0 to 7031
Data columns (total 21 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerID                 7032 non-null   string 
 1   Churn                      7032 non-null   bool   
 2   customer_gender            7032 non-null   string 
 3   customer_SeniorCitizen     7032 non-null   bool   
 4   customer_Partner           7032 non-null   bool   
 5   customer_Dependents        7032 non-null   bool   
 6   customer_tenure            7032 non-null   int64  
 7   phone_PhoneService         7032 non-null   bool   
 8   phone_MultipleLines        7032 non-null   bool   
 9   internet_InternetService   7032 non-null   string 
 10  internet_OnlineSecurity    7032 non-null   bool   
 11  internet_OnlineBackup      7032 non-null   bool   
 12  internet_DeviceProtection  7032 non-null   bool   
 13  internet_TechSupport       7032 non-null   bool 

#📊 Carga y análisis

In [425]:
flatt_DF.sample(5)

,customerID,Churn,customer_gender,customer_SeniorCitizen,customer_Partner,customer_Dependents,customer_tenure,phone_PhoneService,phone_MultipleLines,internet_InternetService,...,internet_OnlineBackup,internet_DeviceProtection,internet_TechSupport,internet_StreamingTV,internet_StreamingMovies,account_Contract,account_PaperlessBilling,account_PaymentMethod,account_Charges.Monthly,account_Charges.Total
4110,5639-NTUPK,No,Male,0,No,Yes,13,Yes,No,DSL,...,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),52.10,670.65
1066,1493-AMTIE,Yes,Male,0,No,No,1,Yes,No,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Bank transfer (automatic),20.20,20.2
5121,6994-FGRHH,No,Male,0,Yes,Yes,70,Yes,No,Fiber optic,...,Yes,Yes,Yes,Yes,Yes,One year,Yes,Electronic check,106.15,7475.1
5168,7054-ENNGU,No,Female,0,Yes,No,9,Yes,No,DSL,...,Yes,No,No,No,No,Month-to-month,No,Mailed check,50.85,466.6
4064,5566-SOEZD,No,Male,0,Yes,Yes,27,Yes,No,Fiber optic,...,Yes,No,No,No,No,One year,Yes,Credit card (automatic),80.65,2209.75


In [426]:
flatt_DF['account_Charges.Total']


,account_Charges.Total
0,593.3
1,542.4
2,280.85
3,1237.85
4,267.4
...,...
7262,742.9
7263,1873.7
7264,92.75
7265,4627.65


#📄Informe final